# Develop negative log-likelihood with scaled inputs

In [ ]:
from __future__ import annotations

import numdifftools as nd
import numpy as np

from numpy.typing import ArrayLike
from scipy.fft import rfft, irfft, rfftfreq
from scipy.optimize import minimize
from matplotlib import pyplot as plt
from matplotlib.figure import figaspect
from scipy.optimize import approx_fprime
from numpy.random import default_rng

import thztools as thz
from thztools.thztools import _costfun_noisefit as costfun

## Simulate measurements

In [ ]:
rng = np.random.default_rng(0)
n = 256
m = 64
dt = 0.05
thz.set_option("sampling_time", dt)
t = thz.timebase(n)
mu = thz.wave(n)
sigma_alpha, sigma_beta, sigma_tau = 1e-5, 1e-2, 1e-3
noise_model = thz.NoiseModel(
    sigma_alpha=sigma_alpha, sigma_beta=sigma_beta, sigma_tau=sigma_tau
)
noise = noise_model.noise_sim((np.ones((m, 1)) * mu), seed=0)
x = np.array(mu + noise)
delta_mu = np.zeros(n)
delta_a = np.zeros(m - 1)
eta = np.zeros(m - 1)

scale_logv_alpha = 1 / m
scale_logv_beta = 1 / m
scale_logv_tau = 1 / m
scale_logv = np.array([scale_logv_alpha, scale_logv_beta, scale_logv_tau])

scale_delta_mu = 1e-0 * noise_model.noise_amp(mu)
scale_delta_a = 1e-4 * np.ones(m - 1)
scale_eta = 1e-3 * np.ones(m - 1)

logv_scaled = np.array(
    [
        np.log(sigma_alpha**2) / scale_logv_alpha,
        np.log(sigma_beta**2) / scale_logv_beta,
        np.log((sigma_tau / dt) ** 2) / scale_logv_tau,
    ]
)
print(f"{logv_scaled=}")

In [ ]:
d_var = ((x - mu) ** 2 - np.var(x, axis=0)) / np.var(x, axis=0) ** 2
mu_f = np.fft.rfft(mu)
w = 2 * np.pi * np.fft.rfftfreq(n, dt)
dmu_dt = np.fft.irfft(1j * w * mu_f, n=n)
[np.sum(d_var), np.sum(d_var * mu**2), np.sum(d_var * dmu_dt**2)] / np.sum(
    d_var
)

In [ ]:
result = thz.noisefit(
    x.T,
    sigma_alpha0=sigma_alpha,
    sigma_beta0=sigma_beta,
    sigma_tau0=sigma_tau,
    mu0=mu,
    a0=1 + np.concatenate(([0.0], delta_a)),
    eta0=np.concatenate(([0.0], eta)),
    dt=dt,
    fix_sigma_alpha=False,
    fix_sigma_beta=False,
    fix_sigma_tau=False,
    fix_mu=False,
    fix_a=False,
    fix_eta=False,
    scale_logv_alpha=scale_logv_alpha,
    scale_logv_beta=scale_logv_beta,
    scale_logv_tau=scale_logv_tau,
    scale_delta_mu=scale_delta_mu,
    scale_delta_a=scale_delta_a,
    scale_eta=scale_eta,
)
p_opt = result.diagnostic.x

In [ ]:
print(result.diagnostic["message"])

In [ ]:
sigma = np.array([sigma_alpha, sigma_beta, sigma_tau])

sigma_out = np.array(
    [
        result.noise_model.sigma_alpha,
        result.noise_model.sigma_beta,
        result.noise_model.sigma_tau,
    ]
) * np.sqrt(m / (m - 1))
sigma_err = np.array(
    [result.err_sigma_alpha, result.err_sigma_beta, result.err_sigma_tau]
) * np.sqrt(m / (m - 1))
for _in, _out, _err in zip(sigma, sigma_out, sigma_err):
    print(f"Input: {_in:6.4g}\t Output: {_out:6.4g} ± {_err:6.4g}")

In [ ]:
sigma_out

In [ ]:
np.diag(result.diagnostic.hess_inv[:3, :3])

## Check gradient

In [ ]:
_, grad_delta_mu_tdnll = costfun(
    x,
    logv_scaled[0],
    logv_scaled[1],
    logv_scaled[2],
    delta_mu / scale_delta_mu,
    delta_a / scale_delta_a,
    eta / scale_eta,
    fix_logv_alpha=True,
    fix_logv_beta=True,
    fix_logv_tau=True,
    fix_delta_mu=False,
    fix_delta_a=True,
    fix_eta=True,
    scale_logv_alpha=scale_logv_alpha,
    scale_logv_beta=scale_logv_beta,
    scale_logv_tau=scale_logv_tau,
    scale_delta_mu=scale_delta_mu,
    scale_delta_a=scale_delta_a,
    scale_eta_on_dt=scale_eta / dt,
)

grad_delta_mu_nd = nd.Gradient(
    lambda _delta_mu: costfun(
        x,
        logv_scaled[0],
        logv_scaled[1],
        logv_scaled[2],
        _delta_mu,
        delta_a / scale_delta_a,
        eta / scale_eta,
        fix_logv_alpha=True,
        fix_logv_beta=True,
        fix_logv_tau=True,
        fix_delta_mu=True,
        fix_delta_a=True,
        fix_eta=True,
        scale_logv_alpha=scale_logv_alpha,
        scale_logv_beta=scale_logv_beta,
        scale_logv_tau=scale_logv_tau,
        scale_delta_mu=scale_delta_mu,
        scale_delta_a=scale_delta_a,
        scale_eta_on_dt=scale_eta / dt,
    )[0],
)(delta_mu / scale_delta_mu)

np.stack(
    (
        grad_delta_mu_tdnll,
        grad_delta_mu_nd,
    )
).T

In [ ]:
plt.plot(t, grad_delta_mu_tdnll)
plt.plot(t, grad_delta_mu_nd)
plt.show()
plt.plot(t, grad_delta_mu_tdnll - grad_delta_mu_nd)
plt.show()

In [ ]:
_, grad_logv_tdnll = costfun(
    x,
    logv_scaled[0],
    logv_scaled[1],
    logv_scaled[2],
    delta_mu / scale_delta_mu,
    delta_a / scale_delta_a,
    eta / scale_eta,
    fix_logv_alpha=False,
    fix_logv_beta=False,
    fix_logv_tau=False,
    fix_delta_mu=True,
    fix_delta_a=True,
    fix_eta=True,
    scale_logv_alpha=scale_logv_alpha,
    scale_logv_beta=scale_logv_beta,
    scale_logv_tau=scale_logv_tau,
    scale_delta_mu=scale_delta_mu,
    scale_delta_a=scale_delta_a,
    scale_eta_on_dt=scale_eta / dt,
)

grad_logv_nd = nd.Gradient(
    lambda _logv: costfun(
        x,
        _logv[0],
        _logv[1],
        _logv[2],
        delta_mu / scale_delta_mu,
        delta_a / scale_delta_a,
        eta / scale_eta,
        fix_logv_alpha=True,
        fix_logv_beta=True,
        fix_logv_tau=True,
        fix_delta_mu=True,
        fix_delta_a=True,
        fix_eta=True,
        scale_logv_alpha=scale_logv_alpha,
        scale_logv_beta=scale_logv_beta,
        scale_logv_tau=scale_logv_tau,
        scale_delta_mu=scale_delta_mu,
        scale_delta_a=scale_delta_a,
        scale_eta_on_dt=scale_eta / dt,
    )[0]
)(logv_scaled)

np.stack(
    (
        grad_logv_tdnll,
        grad_logv_nd,
    )
).T

In [ ]:
_, grad_delta_a_tdnll = costfun(
    x,
    logv_scaled[0],
    logv_scaled[1],
    logv_scaled[2],
    delta_mu / scale_delta_mu,
    delta_a / scale_delta_a,
    eta / scale_eta,
    fix_logv_alpha=True,
    fix_logv_beta=True,
    fix_logv_tau=True,
    fix_delta_mu=True,
    fix_delta_a=False,
    fix_eta=True,
    scale_logv_alpha=scale_logv_alpha,
    scale_logv_beta=scale_logv_beta,
    scale_logv_tau=scale_logv_tau,
    scale_delta_mu=scale_delta_mu,
    scale_delta_a=scale_delta_a,
    scale_eta_on_dt=scale_eta / dt,
)

grad_delta_a_nd = nd.Gradient(
    lambda _delta_a: costfun(
        x,
        logv_scaled[0],
        logv_scaled[1],
        logv_scaled[2],
        delta_mu / scale_delta_mu,
        _delta_a,
        eta / scale_eta,
        fix_logv_alpha=True,
        fix_logv_beta=True,
        fix_logv_tau=True,
        fix_delta_mu=True,
        fix_delta_a=True,
        fix_eta=True,
        scale_logv_alpha=scale_logv_alpha,
        scale_logv_beta=scale_logv_beta,
        scale_logv_tau=scale_logv_tau,
        scale_delta_mu=scale_delta_mu,
        scale_delta_a=scale_delta_a,
        scale_eta_on_dt=scale_eta / dt,
    )[0]
)(delta_a)

np.stack((grad_delta_a_tdnll, grad_delta_a_nd)).T

In [ ]:
plt.plot(grad_delta_a_tdnll)
plt.plot(grad_delta_a_nd)
plt.show()
plt.plot(grad_delta_a_tdnll - grad_delta_a_nd)
plt.show()

In [ ]:
_, grad_eta_tdnll = costfun(
    x,
    logv_scaled[0],
    logv_scaled[1],
    logv_scaled[2],
    delta_mu / scale_delta_mu,
    delta_a / scale_delta_a,
    eta / scale_eta,
    fix_logv_alpha=True,
    fix_logv_beta=True,
    fix_logv_tau=True,
    fix_delta_mu=True,
    fix_delta_a=True,
    fix_eta=False,
    scale_logv_alpha=scale_logv_alpha,
    scale_logv_beta=scale_logv_beta,
    scale_logv_tau=scale_logv_tau,
    scale_delta_mu=scale_delta_mu,
    scale_delta_a=scale_delta_a,
    scale_eta_on_dt=scale_eta / dt,
)

grad_eta_nd = nd.Gradient(
    lambda _eta_scaled: costfun(
        x,
        logv_scaled[0],
        logv_scaled[1],
        logv_scaled[2],
        delta_mu / scale_delta_mu,
        delta_a / scale_delta_a,
        _eta_scaled,
        fix_logv_alpha=True,
        fix_logv_beta=True,
        fix_logv_tau=True,
        fix_delta_mu=True,
        fix_delta_a=True,
        fix_eta=True,
        scale_logv_alpha=scale_logv_alpha,
        scale_logv_beta=scale_logv_beta,
        scale_logv_tau=scale_logv_tau,
        scale_delta_mu=scale_delta_mu,
        scale_delta_a=scale_delta_a,
        scale_eta_on_dt=scale_eta / dt,
    )[0]
)(eta)

np.stack((grad_eta_tdnll, grad_eta_nd)).T

In [ ]:
plt.plot(grad_eta_tdnll)
plt.plot(grad_eta_nd)
plt.show()
plt.plot(grad_eta_tdnll - grad_eta_nd)
plt.show()

In [ ]:
_, grad_all_tdnll = costfun(
    x,
    logv_scaled[0],
    logv_scaled[1],
    logv_scaled[2],
    delta_mu / scale_delta_mu,
    delta_a / scale_delta_a,
    eta / scale_eta,
    fix_logv_alpha=False,
    fix_logv_beta=False,
    fix_logv_tau=False,
    fix_delta_mu=False,
    fix_delta_a=False,
    fix_eta=False,
    scale_logv_alpha=scale_logv_alpha,
    scale_logv_beta=scale_logv_beta,
    scale_logv_tau=scale_logv_tau,
    scale_delta_mu=scale_delta_mu,
    scale_delta_a=scale_delta_a,
    scale_eta_on_dt=scale_eta / dt,
)

p_all = np.concatenate(
    (
        logv_scaled,
        delta_mu / scale_delta_mu,
        delta_a / scale_delta_a,
        eta / scale_eta,
    )
)
grad_all_nd = nd.Gradient(
    lambda _p: costfun(
        x,
        _p[0],
        _p[1],
        _p[2],
        _p[3 : 3 + n],
        _p[3 + n : 3 + n + m - 1],
        _p[3 + n + m - 1 : 3 + n + m - 1 + m - 1],
        fix_logv_alpha=True,
        fix_logv_beta=True,
        fix_logv_tau=True,
        fix_delta_mu=True,
        fix_delta_a=True,
        fix_eta=True,
        scale_logv_alpha=scale_logv_alpha,
        scale_logv_beta=scale_logv_beta,
        scale_logv_tau=scale_logv_tau,
        scale_delta_mu=scale_delta_mu,
        scale_delta_a=scale_delta_a,
        scale_eta_on_dt=scale_eta / dt,
    )[0]
)(p_all)

np.stack((grad_all_tdnll, grad_all_nd)).T

## Estimate noise parameters with revised NLL

In [ ]:
grad_all_opt_nd = nd.Gradient(
    lambda _p: costfun(
        x,
        _p[0],
        _p[1],
        _p[2],
        _p[3 : 3 + n],
        _p[3 + n : 3 + n + m - 1],
        _p[3 + n + m - 1 : 3 + n + m - 1 + m - 1],
        fix_logv_alpha=True,
        fix_logv_beta=True,
        fix_logv_tau=True,
        fix_delta_mu=True,
        fix_delta_a=True,
        fix_eta=True,
        scale_logv_alpha=scale_logv_alpha,
        scale_logv_beta=scale_logv_beta,
        scale_logv_tau=scale_logv_tau,
        scale_delta_mu=scale_delta_mu,
        scale_delta_a=scale_delta_a,
        scale_eta_on_dt=scale_eta / dt,
    )[0]
)(p_opt)

In [ ]:
plt.plot(result.diagnostic["jac"])
plt.plot(grad_all_opt_nd)
plt.show()

plt.plot(result.diagnostic["jac"] - grad_all_opt_nd)
plt.show()

In [ ]:
plt.semilogy(np.diag(result.diagnostic["hess_inv"]))
plt.show()

In [ ]:
np.diag(result.diagnostic.hess_inv[:3, :3])

$M = 64, N = 256$: array([4.02152611e+00, 1.25599933e+03, 1.00000000e+00])

$M = 32, N = 256$: array([0.00033427, 0.00142627, 0.01256625])

In [ ]:
1 / (
    (m / 2)
    * sigma_alpha**2
    * np.sum(((x - mu) ** 2 - np.var(x, axis=0)) / np.var(x, axis=0) ** 2)
)

In [ ]:
1 / (
    (m / 2)
    * sigma_beta**2
    * np.sum(
        mu**2 * ((x - mu) ** 2 - np.var(x, axis=0)) / np.var(x, axis=0) ** 2
    )
)

In [ ]:
mu_f = np.fft.rfft(mu)
w = 2 * np.pi * np.fft.rfftfreq(n, dt)
dmu_dt = np.fft.irfft(1j * w * mu_f, n=n)

In [ ]:
1 / (
    (m / 2)
    * sigma_tau**2
    * np.sum(
        dmu_dt**2
        * ((x - mu) ** 2 - np.var(x, axis=0))
        / np.var(x, axis=0) ** 2
    )
)